### WEATHER API
  Instructions:
OpenWeather API
Create an account here at: https://openweathermap.org/ . You will use their API to get weather forecast data
After you have created the free account, request an API key. The key enables you to connect to their API and pull weather data.

  Weather Function
Create a function which will make requests to the OpenWeather Current Weather Data API to get weather information using the latitude and longitude coordinates for a city.
Parse the response from the API request and extract the following information about a city: id,name,country,main description,temperature,minimum temperature,maximum temperature,feels_like,humidity and wind speed.

The function should return a dictionary of the format: {
"id": 802,
"city": "Skopje",
"latitude": 42,
"longtitude": 21.4333,
"country": 25,
"description": "scattered clouds",
"temp":290.97,
"temp_min": 293.97,
"temp_max": 293.97,
"feels_like": 293.23,
"humidity": 43,
"wind_speed": 2.06
}
Design the function in a way that it should work for any given pair of values for latittude and longitute coordinates.
Handle failures in weather function

A failure in this case as an example would be considered passing wrong values for latitude and longtitude. For this reason, modify the weather function you developed before to gracefully fail if wrong values for latitude and longtitude are passed.

The function should capture the response error message from the API call. In these cases, record this information as a new entry into the dataframe . Explore the structure of this dataframe to see what information you need to capture and what columns you need to populate. If required, process the response further to populate the columns accordingly.

In [102]:
import pandas as pd

dfCities = pd.read_csv('../data/worldcities.csv')

latitude = dfCities['lat']
longitude = dfCities['lng']

print(latitude, longitude)


0        35.6870
1        -6.1750
2        28.6100
3        23.1300
4        19.0761
          ...   
48054    14.7040
48055   -25.4745
48056   -33.9681
48057   -18.9315
48058    18.3420
Name: lat, Length: 48059, dtype: float64 0        139.7495
1        106.8275
2         77.2300
3        113.2600
4         72.8775
           ...   
48054     43.5990
48055     30.9703
48056     25.5981
48057     27.8070
48058    -64.9331
Name: lng, Length: 48059, dtype: float64


In [108]:
import requests
import pandas as pd

dfCities = pd.read_csv('../data/worldcities.csv')
dfCities = dfCities.head(10)

successRequests = []
failedRequests = []

APIkey = '3b44ca87fb86045dc73d251817916e47'
#lat = '42.0062'
#lon = '20.9698'
#r = requests.get(f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={APIkey}')  #forecast api
#r = requests.get(f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={APIkey}')    #weather api


#print(r.country)
#print(r.url)
#print(r.status_code)
#print(r.text)
#data = r.json()


def getWeather(lat, lon, APIkey):

    try:
        r = requests.get(f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={APIkey}&units=metric') #to get in celsius &units=metric
        r.raise_for_status() #jump to except code
        weatherData = r.json()

        weather = {
            "id": weatherData['id'],
            "city": weatherData['name'],
            "latitude": weatherData['coord']['lat'],
            "longitude": weatherData['coord']['lon'], # Spelling from your prompt
            "country": weatherData['sys']['country'],
            "description": weatherData['weather'][0]['description'], # 'weather' is a list!
            "temp": weatherData['main']['temp'],
            "temp_min": weatherData['main']['temp_min'],
            "temp_max": weatherData['main']['temp_max'],
            "feels_like": weatherData['main']['feels_like'],
            "humidity": weatherData['main']['humidity'],
            "wind_speed": weatherData['wind']['speed']
        }

        return(weather)

    except requests.exceptions.HTTPError as e:
        errorData = e.response.json()

        error = {
            'Error Code': errorData['cod'],
            'Error Message': errorData['message'],
            'latitute': lat,
            'longitude': lon
        }
        return (errorData, error)

getWeather(42,  21.4333, APIkey)



latlonDict = [
    (42, 21.4333),
    (42,24),
    (41,41),
    (999, 53),
    (999,999),
    (41, -500)
]

for lat, lon in latlonDict:
    result = getWeather(lat, lon, APIkey)

    if isinstance(result, tuple):
        errorData = result[1]
        failedRequests.append(errorData)

    else:
        successRequests.append(result)



for index, row in dfCities.iterrows():
    lat = row['lat']
    lon = row['lng']

    weatherInfo = getWeather(lat, lon, APIkey)

    successRequests.append(weatherInfo)

df = pd.DataFrame(successRequests)
dfErrors = pd.DataFrame(failedRequests)

#getWeather(999, 42, APIkey)

#getWeather(999, 9999, APIkey)

print(df)
print(dfErrors)

df.to_csv(f"../output/apiRequest.csv",index=False)
dfErrors.to_csv(f"../output/errorRequests.csv",index=False)




         id                   city  latitude  longitude country  \
0    785842                 Skopje   41.9961    21.4316      MK   
1    729519               Ladzhene   42.0000    24.0000      BG   
2    749841            Çamlıhemşin   41.0000    41.0000      TR   
3   1861060                  Japan   35.6870   139.7495      JP   
4   1631845             Pecenongan   -6.1750   106.8275      ID   
5   1261481              New Delhi   28.6100    77.2300      IN   
6   1809858              Guangzhou   23.1300   113.2600      CN   
7   8131499        Konkan Division   19.0818    72.8795      IN   
8   1688216             Santa Cruz   14.6009   120.9838      PH   
9   1796231  Shanghai Municipality   31.2286   121.4747      CN   
10  3458611              Liberdade  -23.5504   -46.6339      BR   
11  1835848                  Seoul   37.5667   126.9833      KR   
12  3530597            Mexico City   19.4333   -99.1333      MX   

         description   temp  temp_min  temp_max  feels_like  

In [ ]:
id = data['id']
name = data['name']
country = data['sys']['country']
mainDescription = data['weather'][0]['description']
temperature = data['main']['temp']
minTemp = data['main']['temp_min']
maxTemp = data['main']['temp_max']
feelsLike = data['main']['feels_like']
humidity = data['main']['humidity']
windSpeed = data['wind']['speed']

print(id, name, country, mainDescription, temperature, minTemp, maxTemp, feelsLike, humidity, windSpeed)

#print(r.json())

#print(data['list'][0]['weather'][0]['description'])
#print(data['list'][0]['temp'])
#print(data['city'])